batch normalization 有利于训练深层神经网络，持续增加深层网络的收敛速度  
猜想：在反向传播的过程中，由于网络的深度，导致了靠近输出层的梯度比较大（因为LOSS原始数据也大），靠近输入的层梯度越小。  
当lr一定时，输入层因为梯度比较小，更新更少，但是输出层的梯度是很大的，所以输出层会很快收敛。  
但是如果输入层的参数更新了之后（因为不同的batch里面的数据的分布可能不同），输出层的参数又要重新学习（以适应不同的分布），导致网络整体的参数收敛的速度变慢。  
> 需求：在更换batch，输入层参数更新的时候避免更改输出层的参数  
解决方法：由于我们本质上就是在学习数据的分布，所以如果能够把每一个batch的数据都能固定下来，就能更容易学习不同batches之间的细微变化  
* 固定下小批量的均值和方差
$$ {\mu }_{B}=\frac {1} {|B|}\sum _{i\in B} {{x}_{i}}\, ,\, {\sigma }^{2}_{B}=\frac {1} {|B|}\sum _{i\in B} {{({x}_{i}-{\mu }_{B})}^{2}}+\epsilon  $$  

* 加上可学习的参数构成最终的层：  
$$ {x}_{i+1}=\gamma \frac {{x}_{i}-{\mu }_{B}} {{\sigma }_{B}}+\beta  $$  
可学习的参数: $ \gamma $、$ \beta $,NB层作用在全联接和卷积的输出上、激活函数之前，对于全联接层，作用在特征维，对于卷积层，作用在通道维（当作是卷积层的一个特征维）

有论文指出，这个东西可能就是在以恶搞小批量里面增加噪音（随机的一个均值和方差）来控制模型的复杂度，没必要和DropOut混用